In [1]:
!nvidia-smi

Sat Sep 30 17:09:36 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A4000                On | 00000000:01:00.0 Off |                  Off |
| 41%   41C    P8               14W / 140W|     10MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/manidhar/Desktop/AryanWork/yolov8-seg-training


In [3]:
from ultralytics import YOLO


MODEL = "Trained_Models/nano_segment_model.pt"
model = YOLO(MODEL) 

In [4]:

def predict_raised_axle(axle_data,threshhold):

    import numpy as np

    if(len(axle_data)==0):
        return False
    if(len(axle_data)==5):
        return True

    has_raised_axle = False
    slopes = []
    axle_count = len(axle_data)-1
    
    np.sort(axle_data)


    for i in range(0,axle_count):
        slopes.append(int(-1)*(axle_data[axle_count][0]-axle_data[i][0])/(axle_data[axle_count][1]-axle_data[i][1]))

    # print(slopes)
    std_dev = np.std(slopes)

    if(std_dev>=threshhold):
        has_raised_axle = True

    return has_raised_axle


    

In [5]:
def get_axle_inside_bounding_box(bounding_box,axle_data): # both in [x,y,x,y] format

    axles_inside_bounding_box = []

    bounding_box_x_top = bounding_box[0]
    bounding_box_y_top = bounding_box[1]
    bounding_box_x_bot = bounding_box[2]
    bounding_box_y_bot = bounding_box[3] 


    for data in axle_data:
        x_top = int(data[0])
        y_top = int(data[1])

        if(bounding_box_x_top<=x_top and x_top<=bounding_box_x_bot and bounding_box_y_top<=y_top and y_top<=bounding_box_y_bot):
            axles_inside_bounding_box.append([(data[0]+data[2])/2,(data[1]+data[3])/2])
    
    return axles_inside_bounding_box

In [6]:
# {0: 'A-10-S-TANDEM', 1: 'A-10-TRIDEM', 2: 'AXLE', 3: 'COMBINED', 4: 'TYPE-2', 5: 'TYPE-2-S2', 6: 'TYPE-3', 7: 'UC'}

# modified

def contains_raised_axle(filepath_of_image,threshold):

    import numpy

    result = model(filepath_of_image)
    predicted_boxes = []
    predicted_classes = []
    axles = []
    axles_inside_bounding_box = []

    if len(result[0]) !=0:
        predicted_boxes = result[0].boxes.xyxy.cpu().numpy()
        predicted_classes= result[0].boxes.cls.cpu().numpy()

    for classes in range(len(predicted_classes)):
        if(predicted_classes[classes]==2):
            axles.append(predicted_boxes[classes])

    for classes in range(len(predicted_classes)):
        if(predicted_classes[classes] == 3):
            axles_inside_bounding_box = get_axle_inside_bounding_box(predicted_boxes[classes],axles)

    return predict_raised_axle(axles_inside_bounding_box,threshold)



In [7]:
def get_file_name_of_image_without_extension(image_path):
    import os
    image_filename_without_ext = os.path.splitext(os.path.basename(image_path))[0]
    return image_filename_without_ext

In [8]:

def get_lifted_axle_images(path_to_dataset):
    
    import glob
    import numpy as np


    slope_threshold = 0.1
    images_with_raised_axle = {" "}

    for image_path in glob.glob(HOME + path_to_dataset + '/test/images/*.jpg')[:]:
        if(contains_raised_axle(image_path,slope_threshold)):
            images_with_raised_axle.add(get_file_name_of_image_without_extension(image_path))
            # model.predict(image_path,save=True,hide_labels=True)


    for images in images_with_raised_axle:
        print(images)
    
    print(len(images_with_raised_axle))

In [9]:
path_to_dataset = '/MASTERDATASET'

get_lifted_axle_images(path_to_dataset)

Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
YOLOv8n-seg summary (fused): 195 layers, 3259624 parameters, 0 gradients, 12.0 GFLOPs


 
gjftuyr6f_jpg.rf.69eb480486788251469d8b86efda35b5
2023-06-03-09_12_13-620204__00000005_jpg.rf.aec5114f0354b1c4f0dff23dd1bff9a3
mainak-1_211_jpg.rf.37fd2f72e4cf2a088807ef796a649d3d
mainak-3_593_jpg.rf.1fbf57fe64d52cee47eda86670e16b30
2023-06-03-06_31_58-886433__00000015_jpg.rf.056817d58abde5ff7950aa611bfbbcac
mainak-3_198_jpg.rf.6d9bec74eb2c786bcb5fd2879ef90e64
sravya-14_08_47-766729__00000014_jpg.rf.b0429e0f472d85d152cb0a5888c30934
mainak-3_178_jpg.rf.bdbb539e2ceba0078c71deef609e3c63
9


In [10]:
# A - 13 Test Image 2023-06-03-01_30_16-745497__00000007_jpg.rf.b73b73f7667f37c2d19e53b84ab4e205.jpg

contains_raised_axle("MASTERDATASET/test/images/2023-06-03-01_30_16-745497__00000007_jpg.rf.b73b73f7667f37c2d19e53b84ab4e205.jpg",0.3)

False

In [11]:
# Type-3-S2 Test Image mainak-3_198_jpg.rf.9ab83c35318e7699f6fc0446239d395b.jpg

contains_raised_axle("MASTERDATASET/test/images/mainak-3_198_jpg.rf.6d9bec74eb2c786bcb5fd2879ef90e64.jpg",0.3)

True

In [12]:
model.predict("MASTERDATASET/test/images/mainak-1_211_jpg.rf.37fd2f72e4cf2a088807ef796a649d3d.jpg",save=True,hide_labels=True)

Results saved to runs/segment/predict4


[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([7, 6])
 dtype: torch.float32
  + tensor([[9.00000e+00, 1.71000e+02, 3.05000e+02, 4.68000e+02, 9.33088e-01, 3.00000e+00],
         [1.96000e+02, 3.30000e+02, 2.14000e+02, 3.80000e+02, 8.71050e-01, 2.00000e+00],
         [8.90000e+01, 4.10000e+02, 1.17000e+02, 4.67000e+02, 8.68567e-01, 2.00000e+00],
         [1.69000e+02, 3.51000e+02, 1.90000e+02, 4.03000e+02, 8.49589e-01, 2.00000e+00],
         [2.66000e+02, 2.70000e+02, 2.80000e+02, 3.11000e+02, 8.04015e-01, 2.00000e+00],
         [2.80000e+02, 2.59000e+02, 2.94000e+02, 2.97000e+02, 7.61923e-01, 2.00000e+00],
         [2.50000e+02, 2.76000e+02, 2.64000e+02, 3.19000e+02, 3.63929e-01, 2.00000e+00]], device='cuda:0')Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Masks'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([7, 640, 640])
 dtype: torch.float32
  + tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
    

In [13]:
model.predict("MASTERDATASET/test/images/2023-06-03-01_30_16-745497__00000007_jpg.rf.b73b73f7667f37c2d19e53b84ab4e205.jpg",save=True,hide_labels=True)

Results saved to runs/segment/predict4


[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([7, 6])
 dtype: torch.float32
  + tensor([[ 43.00000, 437.00000, 231.00000, 558.00000,   0.99362,   3.00000],
         [ 95.00000, 522.00000, 113.00000, 559.00000,   0.81833,   2.00000],
         [161.00000, 494.00000, 175.00000, 526.00000,   0.81003,   2.00000],
         [144.00000, 503.00000, 158.00000, 536.00000,   0.79201,   2.00000],
         [205.00000, 476.00000, 216.00000, 504.00000,   0.78905,   2.00000],
         [217.00000, 471.00000, 227.00000, 498.00000,   0.76277,   2.00000],
         [191.00000, 481.00000, 203.00000, 511.00000,   0.69014,   2.00000]], device='cuda:0')Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Masks'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([7, 640, 640])
 dtype: torch.float32
  + tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
    